# 第六部分 元编程 19 动态属性和特性

## 19.1 使用动态属性转换数据

在Python中，数据的属性和处理数据的方法统称属性（attribute）。其实，方法只是可调用的属性。除了这二者之外，我们还可以创建特性（property），在不改变类接口的前提下，使用存取方法（即读值方法和设值方法）修改数据属性。


不管服务是由存储还是计算实现的，一个模块提供的所有服务都应该通过统一的方式使用

用户自己定义的类可以通过__getattr__方法实现“虚拟属性”，当访问不存在的属性时（如obj.no_such_attribute），即时计算属性的值

In [ ]:
"""
osconfeed.py: Script to download the OSCON schedule feed

# BEGIN OSCONFEED_DEMO

    >>> feed = load()  # <1>
    >>> sorted(feed['Schedule'].keys())  # <2>
    ['conferences', 'events', 'speakers', 'venues']
    >>> for key, value in sorted(feed['Schedule'].items()):
    ...     print('{:3} {}'.format(len(value), key))  # <3>
    ...
      1 conferences
    484 events
    357 speakers
     53 venues
    >>> feed['Schedule']['speakers'][-1]['name']  # <4>
    'Carina C. Zona'
    >>> feed['Schedule']['speakers'][-1]['serial']  # <5>
    141590
    >>> feed['Schedule']['events'][40]['name']
    'There *Will* Be Bugs'
    >>> feed['Schedule']['events'][40]['speakers']  # <6>
    [3471, 5199]


# END OSCONFEED_DEMO
"""

# BEGIN OSCONFEED
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # <1>
        with urlopen(URL) as remote, open(JSON, 'wb') as local:  # 在with语句中使用两个上下文管理器（从Python 2.7和Python 3.1起允许这么做），分别用于读取和保存远程文件。
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)  # <3>

# END OSCONFEED
feed = load()

### 19.1.1 使用动态属性访问JSON类数据

feed\['Schedule'\]\['events'\]\[40\]\['name'\]这种句法很冗长。在JavaScript中，可以使用feed.Schedule.events\[40\].name获取那个值。在Python中，可以实现一个近似字典的类（网上有大量实现）。

目标：实现一个FrozenJSON类，只支持读取，即只能访问数据。不过，这个类能递归，自动处理嵌套的映射和列表。

In [4]:
# 目标：实现一个FrozenJSON类，只支持读取，即只能访问数据。不过，这个类能递归，自动处理嵌套的映射和列表。

from collections import abc
import keyword

class FrozenJSON:
    """
        一个只读接口,使用属性表示法访问JSON类对象
    """
    def __init__(self, mapping):
        # self.__data = dict(mapping) # 使用mapping参数构建一个字典。这么做有两个目的：(1)确保传入的是字典（或者是能转换成字典的对象）；(2)安全起见，创建一个副本。
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): # 检查是否是python的关键字
                key += '_'
            self.__data[key] = value 

    def __getattr__(self, name): # 仅当没有指定名称（name）的属性时才调用__getattr__方法。(会先调用__getattribute__方法)
        if hasattr(self.__data, name):
            return getattr(self.__data, name) # 如果name是实例属性__data的属性，返回那个属性。调用keys等方法就是通过这种方式处理的。

        else:
            return FrozenJSON.build(self.__data[name]) # 否则，从self.__data中获取name键对应的元素，返回调用FrozenJSON.build（　）方法得到的结果。

    @classmethod
    def build(cls, obj): # 这是一个备选构造方法，@classmethod装饰器经常这么用。
        # Json的数据只有字典和列表两种类型
        if isinstance(obj, abc.Mapping): # 如果obj是映射，那就构建一个FrozenJSON对象。
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): # 如果是MutableSequence对象，必然是列表，因此，我们把obj中的每个元素递归地传给.build（　）方法，构建一个列表。
            return [cls.build(item) for item in obj]
        else: # 如果既不是字典也不是列表，那么原封不动地返回元素。
            return obj
    

In [5]:
# 检查 FrozenJSON 类
import json

filepath = 'chapter19/osconfeed.json'

with open(filepath, 'r') as fp:
    j = json.load(fp)

    feed = FrozenJSON(j)

    print(len(feed.Schedule.speakers))

    print(sorted(feed.Schedule.keys()))

    for key, value in sorted(feed.Schedule.items()):
        print('{:3} {}'.format(len(value), key))
    
    fp.close()

357
['conferences', 'events', 'speakers', 'venues']
  1 conferences
484 events
357 speakers
 53 venues


In [22]:
bf = dir(feed)
events = feed.Schedule.events
aft = dir(feed)

bf == aft # 并没有给feed添加.Schedule.events属性！

True

- 初始化时，将数据保存在__data属性中。
- 使用feed.Schedule.events这种方式取值时，由于本没有该属性，会触发__getattr__()方法来尝试获得。（python获得属性的方法有多种，且有优先级）
- 在__getattr__方法中调用类方法去创建新的类实例作为返回值（并非为原来的实例创建属性！）

### 19.1.3 使用__new__方法以灵活的方式创建对象

其实，用于构建实例的是特殊方法__new__：这是个类方法（使用特殊方式处理，因此不必使用@classmethod装饰器），必须返回一个实例。返回的实例会作为第一个参数（即self）传给__init__方法。因为调用__init__方法时要传入实例，而且禁止返回任何值，所以__init__方法其实是“初始化方法”。真正的构造方法是__new__。

我们几乎不需要自己编写__new__方法，因为从object类继承的实现已经足够了。

骚操作的基础：即从__new__方法到__init__方法，是最常见的，但不是唯一的。__new__方法也可以返回其他类的实例，此时，解释器不会调用__init__方法。

In [23]:
# 使用__new__方法取代build方法，构建可能是也可能不是FrozenJSON实例的新对象

from collections import abc
import keyword

class FrozenJSON2:
    """
        一个只读接口，使用属性表示法访问JSON类对象
    """
    def __new__(cls, obj): # __new__是类方法，第一个参数是类本身，余下的参数与__init__方法一样，只不过没有self。
        if isinstance(obj, abc.Mapping): 
            return super().__new__(cls)  #  默认的行为是委托给超类的__new__方法。这里调用的是object基类的__new__方法，把唯一的参数设为FrozenJSON。
        elif isinstance(obj, abc.MutableSequence):  # __new__方法中余下的代码与原先的build方法完全一样。
            return [cls(item) for item in obj] 
        else: 
            return obj

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): 
                key += '_'
            self.__data[key] = value 

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name) 

        else:
            return FrozenJSON(self.__data[name]) # 之前，这里调用的是FrozenJSON.build方法，现在只需调用FrozenJSON构造方法。

    # 使用__new__内置函数代替：
    # @classmethod
    # def build(cls, obj): # 这是一个备选构造方法，@classmethod装饰器经常这么用。
    #     # Json的数据只有字典和列表两种类型
    #     if isinstance(obj, abc.Mapping): # 如果obj是映射，那就构建一个FrozenJSON对象。
    #         return cls(obj)
    #     elif isinstance(obj, abc.MutableSequence): # 如果是MutableSequence对象，必然是列表，[插图]因此，我们把obj中的每个元素递归地传给.build（　）方法，构建一个列表。
    #         return [cls.build(item) for item in obj]
    #     else: # 如果既不是字典也不是列表，那么原封不动地返回元素。
    #         return obj

In [30]:
filepath = 'chapter19/osconfeed.json'

with open(filepath, 'r') as fp:
    j = json.load(fp)

print(type(j))

j = json.load(fp) # 将无法加载，因为with上下文管理把文件关闭了！

<class 'dict'>


In [24]:
# 检查 FrozenJSON 类
import json

filepath = 'chapter19/osconfeed.json'

with open(filepath, 'r') as fp:
    j = json.load(fp)

    feed = FrozenJSON2(j)

    print(len(feed.Schedule.speakers))

    print(sorted(feed.Schedule.keys()))

    for key, value in sorted(feed.Schedule.items()):
        print('{:3} {}'.format(len(value), key))

    fp.close()

357
['conferences', 'events', 'speakers', 'venues']
  1 conferences
484 events
357 speakers
 53 venues


### 19.1.4 使用shelve模块调整OSCON数据源的结构

接下来的任务是，调整数据结构，以便自动获取所链接的记录。

shelve.open高阶函数返回一个shelve.Shelf实例，这是简单的键值对象数据库，背后由dbm模块支持，具有下述特点。
- shelve.Shelf是abc.MutableMapping的子类，因此提供了处理映射类型的重要方法。
- 此外，shelve.Shelf类还提供了几个管理I/O的方法，如sync和close；
- 它也是一个上下文管理器。
- 只要把新值赋予键，就会保存键和值。
- 键必须是字符串。
- 值必须是pickle模块能处理的对象。



In [1]:
"""
schedule1.py: traversing OSCON schedule data
"""
# BEGIN SCHEDULE1
import warnings
import json

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'


def load_osconfeed():
    with open('chapter19/osconfeed.json', 'r') as fp:
        return json.load(fp)


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  #  这是使用关键字参数传入的属性构建实例的常用简便方式（详情参见下文）。
        # Record.__init__方法展示了一个流行的Python技巧。我们知道，对象的__dict__属性中存储着对象的属性--前提是类中没有声明__slots__属性，如9.8节所述。
        # 因此，更新实例的__dict__属性，把值设为一个映射，能快速地在那个实例中创建一堆属性。另外要处理不能作为属性名的健。

'''
双星号（**): **kwargs  将参数以字典(dict)的形式导入
单星号（*) : *agrs  将所以参数以元组(tuple)的形式导入
'''


def load_db(db):
    raw_data = load_osconfeed()  # 如果本地没有副本，从网上下载JSON数据源。
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合（例如'conferences'、'events'，等等）。
        record_type = collection[:-1]  # record_type的值是去掉尾部's'后的集合名（即把'events'变成'event'）。
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # 使用record_type和'serial'字段构成key。
            record['serial'] = key  # 把'serial'字段的值设为完整的键。
            db[key] = Record(**record)  #  构建Record实例，存储在数据库中的key键名下。

# END SCHEDULE1

import shelve 

with shelve.open(DB_NAME) as db: # shelve.open函数打开现有的数据库文件，或者新建一个。
    if CONFERENCE not in db: # 判断数据库是否填充的简便方法是，检查某个已知的键是否存在；这里检查的键是conference.115，即conference记录（只有一个）的键。
        load_db(db) # 如果数据库是空的，那就调用load_db(db)，加载数据。

    speaker = db['speaker.3471'] # 获取一条speaker记录。
    print(type(speaker)) # 它是示例19-9中定义的Record类的实例。 <class '__main__.Record'>
    print(speaker.name) # 各个Record实例都有一系列自定义的属性，对应于底层JSON记录里的字段。  Anna Martelli Ravenscroft
    print(speaker.twitter) 
    db.close() # 一定要记得关闭shelve.Shelf对象。如果可以，使用with块确保Shelf对象会关闭。（这里没必要手动关闭，因为上下文管理器会自动关闭）

<class '__main__.Record'>
Anna Martelli Ravenscroft
annaraven


### 19.1.5 使用特性获取链接的记录

@property装饰器装饰的方法为特性。

目标是，对于从Shelf对象中获取的event记录来说，读取它的venue或speakers属性时返回的不是编号，而是完整的记录对象。

**Record**

- __init__方法与schedule1.py脚本（见示例19-9）中的一样；为了辅助测试，增加了__eq__方法。

**DbRecord**

- Record类的子类，添加了__db类属性，用于设置和获取__db属性的set_db和get_db静态方法，用于从数据库中获取记录的fetch类方法，以及辅助调试和测试的__repr__实例方法。
- DbRecord.__db类属性的作用是存储打开的shelve.Shelf数据库引用，以便在需要使用数据库的DbRecord.fetch方法及Event.venue和Event.speakers属性中使用。

**Event**

- DbRecord类的子类，添加了用于获取所链接记录的venue和speakers属性，以及特殊的__repr__方法。

![Record Class UML](chapter19/RecordClassUML.png)

In [27]:
import warnings
import inspect # inspect模块在load_db函数中使用
import json

DB_NAME = 'data/schedule2_db' # 因为要存储几个不同类的实例，所以我们要创建并使用不同的数据库文件；这里不用示例19-9中的'schedule1_db'，而是使用'schedule2_db'。
CONFERENCE = 'conference.115'


def load_osconfeed():
    with open('chapter19/osconfeed.json', 'r') as fp:
        return json.load(fp)


def load_db(db):
    raw_data = load_osconfeed()  # 如果本地没有副本，从网上下载JSON数据源。
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合（例如'conferences'、'events'，等等）。
        record_type = collection[:-1]  # record_type的值是去掉尾部's'后的集合名（即把'events'变成'event'）。
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # 使用record_type和'serial'字段构成key。
            record['serial'] = key  # 把'serial'字段的值设为完整的键。
            db[key] = Record(**record)  #  构建Record实例，存储在数据库中的key键名下。


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other): # __eq__方法对测试有重大帮助。
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

# 一个自定义的异常类型
class MissingDatabaseError(RuntimeError):
    """需要数据库，但没有指定数据库时抛出""" 
    # 自定义的异常通常是标志类，没有定义体。写一个文档字符串，说明异常的用途，比只写一个pass语句要好。

 
class DbRecord(Record): # DbRecord类扩展Record类
    __db = None #  __db类属性存储一个打开的shelve.Shelf数据库引用。

    @staticmethod # set_db是静态方法，以此强调不管调用多少次，效果始终一样。
    def set_db(db):
        DbRecord.__db = db # 即使调用Event.set_db(my_db)，__db属性仍在DbRecord类中设置。

    @staticmethod
    def get_db():
        return DbRecord.__db 

    @classmethod # fetch是类方法，因此在子类中易于定制它的行为。
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident] # 从数据库中获取ident键对应的记录。
        except TypeError:
            if db is None: # 如果捕获到TypeError异常，而且db变量的值是None，抛出自定义的异常，说明必须设置数据库。
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else: # 否则，重新抛出TypeError异常，因为我们不知道怎么处理。
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'): # 如果记录有serial属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# 重要的Event类
class Event(DbRecord): # Event类扩展DbRecord类
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial) 
        return self.__class__.fetch(key) # 在venue特性中使用venue_serial属性构建key，然后传给继承自DbRecord类的fetch类方法（详情参见下文）。

    @property
    def speaker(self):
        if not hasattr(self, '_speaker_objs'): # speakers特性检查记录是否有_speaker_objs属性
            spkr_serials = self.__dict__['speakers'] # 如果没有，直接从__dict__实例属性中获取'speakers'属性的值，防止无限递归，因为这个特性的公开名称也是speakers。
            fetch = self.__class__.fetch # 获取fetch类方法的引用（稍后会说明这么做的原因）。
            self._speaker_objs = [  fetch('speaker.{}'.format(key))  # 使用fetch获取speaker记录列表，然后赋值给self._speaker_objs。
                                    for key in spkr_serials]
        return self._speaker_objs # 返回前面获取的列表。

    def __repr__(self):
        if hasattr(self, 'name'): # 如果记录有name属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# --------------------------------------------- RUN ----------------------------------------------------
with shelve.open(DB_NAME) as db: # shelve.open函数打开现有的数据库文件，或者新建一个。
    if CONFERENCE not in db: # 判断数据库是否填充的简便方法是，检查某个已知的键是否存在；这里检查的键是conference.115，即conference记录（只有一个）的键。
        load_db(db) # 如果数据库是空的，那就调用load_db(db)，加载数据。

    DbRecord.set_db(db)

    event = DbRecord.fetch('event.33950')
    
    v = event.venue

    print(event.venue.name)
    for s in event.speaker:
        print(s.name)
    print(event.name)





Portland 251
Anna Martelli Ravenscroft
Alex Martelli
There *Will* Be Bugs


为什么不直接使用self.fetch(key)呢？

哪怕只有一个事件记录有名为'fetch'的键，那么在那个Event实例中，self.fetch获取的是fetch字段的值，而不是Event继承自DbRecord的fetch类方法。这个缺陷不明显，很容易被测试忽略；

In [28]:
import warnings
import inspect # inspect模块在load_db函数中使用
import json

DB_NAME = 'data/schedule2_db' # 因为要存储几个不同类的实例，所以我们要创建并使用不同的数据库文件；这里不用示例19-9中的'schedule1_db'，而是使用'schedule2_db'。
CONFERENCE = 'conference.115'


def load_osconfeed():
    with open('chapter19/osconfeed.json', 'r') as fp:
        return json.load(fp)


def load_db(db):
    raw_data = load_osconfeed()  # 如果本地没有副本，从网上下载JSON数据源。
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # 迭代集合（例如'conferences'、'events'，等等）。
        record_type = collection[:-1]  # record_type的值是去掉尾部's'后的集合名（即把'events'变成'event'）。
        
        cls_name = record_type.capitalize() #  把record_type变量的值首字母变成大写（例如，把'event'变成'Event'），获取可能的类名。
        cls = globals().get(cls_name, DbRecord) # 从模块的全局作用域中获取那个名称对应的对象；如果找不到对象，使用DbRecord
        if inspect.isclass(cls) and issubclass(cls, DbRecord): # 如果获取的对象是类，而且是DbRecord的子类……
            factory = cls # ……把对象赋值给factory变量。因此，factory的值可能是DbRecord的任何一个子类，具体的类取决于record_type的值。
        else:
            factory = DbRecord # 否则，把DbRecord赋值给factory变量。

        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  
            record['serial'] = key  
            db[key] = factory(**record)  #  ……存储在数据库中的对象由factory构建，factory可能是DbRecord类，也可能是根据record_type的值确定的某个子类。


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other): # __eq__方法对测试有重大帮助。
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

# 一个自定义的异常类型
class MissingDatabaseError(RuntimeError):
    """需要数据库，但没有指定数据库时抛出""" 
    # 自定义的异常通常是标志类，没有定义体。写一个文档字符串，说明异常的用途，比只写一个pass语句要好。

 
class DbRecord(Record): # DbRecord类扩展Record类
    __db = None #  __db类属性存储一个打开的shelve.Shelf数据库引用。

    @staticmethod # set_db是静态方法，以此强调不管调用多少次，效果始终一样。
    def set_db(db):
        DbRecord.__db = db # 即使调用Event.set_db(my_db)，__db属性仍在DbRecord类中设置。

    @staticmethod
    def get_db():
        return DbRecord.__db

    @classmethod # fetch是类方法，因此在子类中易于定制它的行为。
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident] # 从数据库中获取ident键对应的记录。
        except TypeError:
            if db is None: # 如果捕获到TypeError异常，而且db变量的值是None，抛出自定义的异常，说明必须设置数据库。
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else: # 否则，重新抛出TypeError异常，因为我们不知道怎么处理。
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'): # 如果记录有serial属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} serial={!r}'.format(cls_name, self.serial)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# 重要的Event类
class Event(DbRecord): # Event类扩展DbRecord类
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial) 
        return self.__class__.fetch(key) # 在venue特性中使用venue_serial属性构建key，然后传给继承自DbRecord类的fetch类方法（详情参见下文）。

    @property
    def speakers(self):
        if not hasattr(self, '_speaker_obs'): # speakers特性检查记录是否有_speaker_objs属性
            spkr_serials = self.__dict__['speakers'] # 如果没有，直接从__dict__实例属性中获取'speakers'属性的值，防止无限递归，因为这个特性的公开名称也是speakers。
            fetch = self.__class__.fetch # 获取fetch类方法的引用（稍后会说明这么做的原因）。
            self._speaker_objs = [  fetch('speaker.{}'.format(key))  # 使用fetch获取speaker记录列表，然后赋值给self._speaker_objs。
                                    for key in spkr_serials]
        return self._speaker_objs # 返回前面获取的列表。

    def __repr__(self):
        if hasattr(self, 'name'): # 如果记录有name属性，在字符串表示形式中使用。
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__() # 否则，调用继承的__repr__方法。


# --------------------------------------------- RUN ----------------------------------------------------
with shelve.open(DB_NAME) as db: # shelve.open函数打开现有的数据库文件，或者新建一个。
    if CONFERENCE not in db: # 判断数据库是否填充的简便方法是，检查某个已知的键是否存在；这里检查的键是conference.115，即conference记录（只有一个）的键。
        load_db(db) # 如果数据库是空的，那就调用load_db(db)，加载数据。

    DbRecord.set_db(db)

    event = DbRecord.fetch('event.33950')
    print(event.venue.name)

    for spkr in event.speakers:
        print('{0.serial} : {0.name}'.format(spkr))

    db.close()


Portland 251
speaker.3471 : Anna Martelli Ravenscroft
speaker.5199 : Alex Martelli


_getattr__方法、hasattr函数、getattr函数、@property装饰器和__dict__属性，来实现动态属性。

特性经常用于把公开的属性变成使用读值方法和设值方法管理的属性，且在不影响客户端代码的前提下实施业务规则

## 19.2 使用特性验证属性

前面只介绍了如何使用@property装饰器实现只读特性。本节要创建一个可读写的特性。



In [29]:
# 已经发布的第一代代码：

class LineItem:
    def __init__(self, description, weight, price):
        self.__description = description
        self.__weight = weight
        self.__price = price

    def subtotal(self):
        return self.__weight * self.__price

In [30]:
# 发现发布的代码有问题，比方weight可以为负。需要修正，但是为了不破话其他代码的引用，我们采用特性来打补丁。

class LineItem:
    def __init__(self, description, weight, price):
        self.__description = description
        self.__weight = weight
        self.__price = price

    def subtotal(self):
        return self.__weight * self.__price

    # 使用特性打补丁
    @property
    def weight(self): # 实现特性的方法，其名称都与公开属性的名称一样——weight。
        return self.__weight

    @weight.setter
    def weight(self, value):
        if value > 0 :
            self.__weight = value
        else:
            raise ValueError('value must be > 0')


In [31]:
# 测试

raisins = LineItem('Golden raisins', 10, 6.95)
print(raisins.subtotal())

raisins.weight = -10.0

print(raisins.subtotal())

69.5


ValueError: value must be > 0

上面虽然修复了重量weight这个属性不能为负，但是价格还是可以为负的。我们当然可以将价格也做成特性，但是这是和weight重复的代码，只是换了个名字而已，因此代码失去了优雅。而去除重复的方法是抽象。

抽象特性的定义有两种方式：
- 特性工厂函数
- 描述符类（更灵活，下章介绍）

## 19.3 特性全解析

虽然内置的property经常用作装饰器，但它其实是一个类。在Python中，函数和类通常可以互换，因为二者都是可调用的对象，而且没有实例化对象的new运算符，所以调用构造方法与调用工厂函数没有区别。此外，只要能返回新的可调用对象，代替被装饰的函数，二者都可以用作装饰器。

In [63]:
# property的经典用法，即使用了这个类的构造函数

class LineItem:
    def __init__(self, description, weight, price):
        self.__description = description
        self.__weight = weight
        self.__price = price

    def subtotal(self):
        return self.__weight * self.__price


    def get_weight(self):
        return self.__weight

    def set_weight(self, value):
        if value > 0 :
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight)

    # 某些情况下，这种经典形式比装饰器句法好；稍后讨论的特性工厂函数就是一例。

LI0 = LineItem('test', 1.0, 2.3)
dir(LI0)


['_LineItem__description',
 '_LineItem__price',
 '_LineItem__weight',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'get_weight',
 'set_weight',
 'subtotal',
 'weight']

### 19.3.1 特性会覆盖实例属性

特性都是类属性，但是特性管理的其实是实例属性的存取。

如果实例和所属的类有同名数据属性，那么实例属性会覆盖（或称遮盖）类属性

本节的主要观点是，obj.attr这样的表达式不会从obj开始寻找attr，而是从obj.__class__开始，而且，仅当类中没有名为attr的特性时，Python才会在obj实例中寻找。这条规则不仅适用于特性，还适用于一整类描述符——覆盖型描述符（overridingdescriptor）。第20章会进一步讨论描述符，那时你会发现，特性其实是覆盖型描述符。

In [35]:
class Class: # 这个类有两个类属性：data数据属性和prop特性。
    data = 'the class data attr'

    @property
    def prop(self):
        return 'the prop value'

obj = Class()
print(vars(obj)) # vars函数返回obj的__dict__属性，表明没有实例属性。
print(obj.data)
obj.data = 'baar' # 为obj.data赋值，创建一个实例属性。
print(vars(obj))
print(obj.data) # 现在读取obj.data，获取的是实例属性的值。从obj实例中读取属性时，实例属性data会遮盖类属性data。
print(Class.data) # Class.data属性的值完好无损。


{}
the class data attr
{'data': 'baar'}
baar
the class data attr


In [36]:
# #实例属性不会遮盖类特性#
print('\n#实例属性不会遮盖类特性#')

print(Class.prop) # 直接从Class中读取prop特性，获取的是特性对象本身，不会运行特性的读值方法。

print(obj.prop) # 读取obj.prop会执行特性的读值方法

# obj.prop = 'foo'  # 因为特性没有设置权限，因此会返回 AtributeError: can't set attribute
obj.__dict__['prop'] = 'foo' # 但是可以直接把'prop'存入obj.__dict__。
print(vars(obj)) # 可以看到，obj现在有两个实例属性：data和prop。

print(obj.prop) # 然而，读取obj.prop时仍会运行特性的读值方法。特性没被实例属性遮盖。

Class.prop = 'baz' # 覆盖Class.prop特性，销毁特性对象。原本是<property object>对象，现在只是个字符串了

print(obj.prop) # 现在，obj.prop获取的是实例属性。Class.prop不是特性了，因此不会再覆盖obj.prop。


#实例属性不会遮盖类特性#
the prop value
{'data': 'baar', 'prop': 'foo'}
the prop value
foo


In [37]:
# #新添的类特性遮盖现有的实例属性#
print("\n#新添的类特性遮盖现有的实例属性#")

print(obj.data) # obj.data获取的是实例属性data。

print(Class.data) # Class.data获取的是类属性data。

Class.data = property(lambda self: 'the "data" prop value') # 使用新特性覆盖Class.data。

print(obj.data) # 现在，obj.data被Class.data特性遮盖了。

del Class.data #  删除特性。

print(obj.data) # 现在恢复原样，obj.data获取的是实例属性data。


#新添的类特性遮盖现有的实例属性#
baar
the class data attr
the "data" prop value
baar


### 19.3.2 特性文档

控制台中的help()函数或IDE等工具需要显示特性的文档时，会从特性的__doc__属性中提取信息。

为特性写文档有两种方式：


In [39]:
# 方式一：
weight = property(get_weight, set_weight, doc='weight in kilograms')

# 方法二：
@property
def bar(self):
    '''The bar attribute'''
    return self.__dict__['bar']

@bar.setter
def bar(self, value):
    self.__dict__['bar'] = value

NameError: name 'get_weight' is not defined

## 19.4 定义一个特性工厂函数



In [64]:
# 使用特性工厂函数！

def quantity(storage_name): # storage_name参数确定各个特性的数据存储在哪儿；对weight特性来说，存储的名称是'weight'。

    def qty_getter(instance): # qty_getter函数的第一个参数可以命名为self，但是这么做很奇怪，因为qty_getter函数不在类定义体中；instance指代要把属性存储其中的LineItem实例。
        return instance.__dict__[storage_name] # qty_getter引用了storage_name，把它保存在这个函数的闭包里；值直接从instance.__dict__中获取，为的是跳过特性，防止无限递归。

    def qty_setter(instance, value): # 定义qty_setter函数，第一个参数也是instance。
        if value > 0 :
            instance.__dict__[storage_name] = value # 值直接存到instance.__dict__中，这也是为了跳过特性。
        else :
            raise ValueError('value must be > 0')
    return property(qty_getter, qty_setter) # 构建一个自定义的特性对象，然后将其返回。


class LineItem:
    weight = quantity('weight') # 使用工厂函数把第一个自定义的特性weight定义为类属性
    price = quantity('price') # 第二次调用，构建另一个自定义的特性，price。
    
    """
        特性是类属性。构建各个quantity特性对象时,要传入LineItem实例属性的名称,让特性管理。
        赋值语句的右边先计算,因此调用quantity()时,weight类属性还不存在。
    """

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight # 这里，特性已经激活，确保不能把weight设为负数或零。
        self.price = price

    def subtotal(self):
        return self.weight * self.price # 这里也用到了特性，使用特性获取实例中存储的值。

    @property
    def prop(self):
        return 'property'

apple = LineItem('This is an apple', 2.3, 1.34)

total = apple.weight * apple.price

p  = apple.prop

# apple.weight = 2.5

print(apple)

print(LineItem.weight)

print(apple.description)

print(type(p))

print(type(apple.weight))

print(vars(apple))


This is an apple
<class 'str'>
<class 'float'>
{'description': 'This is an apple', 'weight': 2.3, 'price': 1.34}


使用传统方式定义特性时，用于存储值的属性名硬编码在读值方法和设值方法中。

这里的qty_getter和qty_setter函数是通用的，要依靠storage_name变量判断从__dict__中获取哪个属性，或者设置哪个属性。每次调用quantity工厂函数构建属性时，都要把storage_name参数设为独一无二的值。

我们使用property对象包装qty_getter和qty_setter函数。需要运行这两个函数时，它们会从闭包中读取storage_name，确定从哪里获取属性的值，或者在哪里存储属性的值。

**工厂函数构建的特性利用了19.3.1节所述的行为：weight特性覆盖了weight实例属性，因此对self.weight或nutmeg.weight的每个引用都由特性函数处理，只有直接存取__dict__属性才能跳过特性的处理逻辑。**

在真实的系统中，分散在多个类中的多个字段可能要做同样的验证，此时最好把quantity工厂函数放在实用工具模块中，以便重复使用。最终可能要重构那个简单的工厂函数，改成更易扩展的描述符类，然后使用专门的子类执行不同的验证。在第20章中，我们会这么做。

## 19.5 处理属性删除操作

使用Python编程时不常删除属性，通过特性删除属性更少见。但是，Python支持这么做

In [12]:
class BlackKnight:
    def __init__(self):
        self.members = ['an arm', 'another arm', 'a leg', 'another leg']
        self.phrases = ["'This but a scratch.", "It's just a flesh wound.", "I'm invincible!", "All right, we'll call it a draw."]

    @property
    def member(self):
        print('next member is:')
        return self.members[0]

    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n--{}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

In [13]:
knight = BlackKnight()

knight.member

next member is:


'an arm'

In [14]:
del knight.member

BLACK KNIGHT (loses an arm)
--'This but a scratch.


In [15]:
del knight.member

BLACK KNIGHT (loses another arm)
--It's just a flesh wound.


## 19.6 处理属性的重要属性和函数

### 19.6.1 影响属性处理方式的特殊属性

```__class__```

对象所属类的引用（即obj.\_\_class\_\_与type(obj)的作用相同）。Python的某些特殊方法，例如__getattr__，只在对象的类中寻找，而不在实例中寻找。

```__dict__```

一个映射，存储对象或类的可写属性。有__dict__属性的对象，任何时候都能随意设置新属性。如果类有__slots__属性，它的实例可能没有__dict__属性。参见下面对__slots__属性的说明。

```__slots__```

类可以定义这个这属性，限制实例能有哪些属性。__slots__属性的值是一个字符串组成的元组，指明允许有的属性。[插图]如果__slots__中没有'__dict__'，那么该类的实例没有__dict__属性，实例只允许有指定名称的属性。

### 19.6.2 处理属性的内置函数

```
dir([object])
```
列出对象的大多数属性。官方文档说，dir函数的目的是交互式使用，因此没有提供完整的属性列表，只列出一组“重要的”属性名。dir函数能审查有或没有__dict__属性的对象。dir函数不会列出__dict__属性本身，但会列出其中的键。dir函数也不会列出类的几个特殊属性，例如__mro__、\_\_bases\_\_和\_\_name\_\_。如果没有指定可选的object参数，dir函数会列出当前作用域中的名称。

```
getattr(object, name[, default])
```

从object对象中获取name字符串对应的属性。获取的属性可能来自对象所属的类或超类。如果没有指定的属性，getattr函数抛出AttributeError异常，或者返回default参数的值（如果设定了这个参数的话）。

```
hasattr(object, name)
```

如果object对象中存在指定的属性，或者能以某种方式（例如继承）通过object对象获取指定的属性，返回True。文档说道：“这个函数的实现方法是调用getattr(object, name)函数，看看是否抛出AttributeError异常。”

```
setattr(object, name, value)
```

把object对象指定属性的值设为value，前提是object对象能接受那个值。这个函数可能会创建一个新属性，或者覆盖现有的属性。
```
vars([object])
```
返回object对象的__dict__属性；如果实例所属的类定义了__slots__属性，实例没有__dict__属性，那么vars函数不能处理那个实例（相反，dir函数能处理这样的实例）。如果没有指定参数，那么vars（　）函数的作用与locals（　）函数一样：返回表示本地作用域的字典。

### 19.6.3 处理属性的特殊方法

使用点号或内置的getattr、hasattr和setattr函数存取属性都会触发下述列表中相应的特殊方法。但是，**直接通过实例的__dict__属性读写属性不会触发这些特殊方法——如果需要，通常会使用这种方式跳过特殊方法。**

要假定特殊方法从类上获取，即便操作目标是实例也是如此。因此，**特殊方法不会被同名实例属性遮盖。**

```
__delattr__(self, name)

```
只要使用del语句删除属性，就会调用这个方法。例如，delobj.attr语句触发Class.\_\_delattr\_\_(obj, 'attr')方法。

```
__dir__(self)
```

把对象传给dir函数时调用，列出属性。例如，dir(obj)触发Class.\_\_dir\_\_(obj)方法。

```
__getattr__(self, name)
```

仅当获取指定的属性失败，搜索过obj、Class和超类之后调用。表达式obj.no_such_attr、getattr(obj, 'no_such_attr')和hasattr(obj, 'no_such_attr')可能会触发Class.\_\_getattr\_\_(obj,'no_such_attr')方法，但是，**仅当在obj、Class和超类中找不到指定的属性时才会触发。**

```
__getattribute__(self, name)
```

尝试获取指定的属性时总会调用这个方法，不过，寻找的属性是特殊属性或特殊方法时除外。点号与getattr和hasattr内置函数会触发这个方法。调用__getattribute__方法且抛出AttributeError异常时，才会调用__getattr__方法。为了在获取obj实例的属性时不导致无限递归，__getattribute__方法的实现要使用super().\_\_getattribute\_\_(obj, name)。

```
__setattr__(self, name, value)
```

尝试设置指定的属性时总会调用这个方法。点号和setattr内置函数会触发这个方法。例如，obj.attr=42和setattr(obj, 'attr',42)都会触发Class.\_\_setattr\_\_(obj, ‘attr’, 42)方法。

> 特殊方法__getattribute__和__setattr__不管怎样都会调用，几乎会影响每一次属性存取，因此比__getattr__方法（只处理不存在的属性名）更难正确使用。与定义这些特殊方法相比，使用特性或描述符相对不易出错。